## Постановка задачи

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

**Промежуточная задача**:  
> Преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача**: 
> Построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

## Импорты

In [ ]:
try:
    import rectools
except:
    %pip install rectools
    import rectools

In [ ]:
import pandas as pd
import numpy as np

import requests
from tqdm.auto import tqdm
from rectools import Columns

## Подготовка данных

### Загрузка

In [ ]:
!mkdir -p ../data/hw2-1/

In [ ]:
import requests
from urllib.parse import urlencode

base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"
public_key = "https://disk.yandex.ru/d/SI1aAooPn9i8TA"

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()["href"]

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open(
    "../data/hw21/likes_data.zip", "wb"
) as f:  # Здесь укажите нужный путь к файлу
    f.write(download_response.content)

In [ ]:
!unzip ../data/hw2-1/likes_data.zip -d ../data/hw2

Archive:  ../data/hw2/likes_data.zip
  inflating: ../data/hw2/likes/baseline.py  
  inflating: ../data/hw2/likes/likes_data.zip  
  inflating: ../data/hw2/likes/score.py  


In [ ]:
!unzip ../data/hw2-1/likes/likes_data.zip -d ../data/hw2

Archive:  ../data/hw2/likes/likes_data.zip
  inflating: ../data/hw2/test        
  inflating: ../data/hw2/__MACOSX/._test  
  inflating: ../data/hw2/track_artists.csv  
  inflating: ../data/hw2/train       
  inflating: ../data/hw2/__MACOSX/._train  


### Препроцессинг

In [ ]:
all_data = []
with open("../data/hw2-1/train", "r") as f:
    for i, line in enumerate(f.readlines()):
        tracks_id = list(map(int, line.split()))
        user_data = np.zeros((len(tracks_id), 2), dtype=int)
        user_data[:, 0] = i
        user_data[:, 1] = tracks_id[::-1]
        all_data.append(user_data)
all_data = np.vstack(all_data)

In [ ]:
data = pd.DataFrame(all_data, columns=[Columns.User, Columns.Item])

In [ ]:
data

,user_id,item_id
0,0,388242
1,0,278503
2,0,102795
3,0,470957
4,0,159637
...,...,...
94188629,1160083,19120
94188630,1160083,326821
94188631,1160083,214132
94188632,1160083,352098


## Реализация функции UsersKFoldPOut

In [ ]:
class UsersKFoldPOut:
    def __init__(self, n_folds, p, random_seed=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed

    def split(self, df):
        ser_num_likes = df.groupby(Columns.User).cumcount()

        users = df[Columns.User].unique()
        users_count = len(users)

        np.random.seed(self.random_seed)
        np.random.shuffle(users)

        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]

            test_mask = df[Columns.User].isin(test_fold_users) & (
                ser_num_likes < self.p
            )
            train_mask = ~df[Columns.User].isin(test_fold_users)

            yield train_mask, test_mask

In [ ]:
p = 1
n_folds = 3
cv = UsersKFoldPOut(n_folds=n_folds, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(data)):
    train = data[train_mask]
    test = data[test_mask]

    s1 = set(train[Columns.User].unique())
    s2 = set(test[Columns.User].unique())

    assert len(s1.intersection(s2)) == 0, "Find common user in two groups"
    assert (
        test.groupby(Columns.User).cumcount().max() <= p
    ), "Find more then p track for user"

    print(f"Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}")

Fold#0 | Train: 62769950, Test: 386695
Fold#1 | Train: 62769950, Test: 386695
Fold#2 | Train: 62770016, Test: 386694
